<a href="https://colab.research.google.com/github/sanket-official/German-audio-pronunciation-browser-based/blob/main/Final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio gtts librosa pydub deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [2]:
!pip install --upgrade openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=ede4fc8bfa0e4b16851930ce8e7692fd5e41e4268692ca60541baf5b9aebd6a9
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [3]:
import whisper
import numpy as np
import librosa
from difflib import SequenceMatcher
from gtts import gTTS
import gradio as gr
import random
from pydub import AudioSegment

In [4]:
# Load Whisper large model for better transcription accuracy
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:52<00:00, 29.0MiB/s]


In [7]:
PHRASES = {
    "Wie geht es Ihnen?": "How are you feeling?",
    "Haben Sie Schmerzen?": "Do you have pain?",
    "Wann haben Sie die Medizin genommen?": "When did you take the medicine?",
    "Ich bin die Krankenschwester.": "I am the nurse.",
    "Atmen Sie tief ein.": "Breathe in deeply."
}

In [17]:
CELEBRATION_GIF = "https://media.giphy.com/media/111ebonMs90YLu/giphy.gif"

def normalize_text(text):
    return text.lower().replace('.', '').replace('?', '').replace('!', '').strip()

def slow_down_audio(input_path, output_path, speed=0.90):
    sound = AudioSegment.from_file(input_path)
    slowed_sound = sound._spawn(sound.raw_data, overrides={
        "frame_rate": int(sound.frame_rate * speed)
    }).set_frame_rate(sound.frame_rate)
    slowed_sound.export(output_path, format="mp3")

def play_audio(phrase):
    tts_path = "native_example.mp3"
    slowed_path = "native_example_slow.mp3"
    tts = gTTS(text=phrase, lang='de', slow=False)
    tts.save(tts_path)
    slow_down_audio(tts_path, slowed_path, speed=0.90)
    return slowed_path


In [18]:
def similarity_score_bar(score):
    green = int((score / 100) * 255)
    red = 255 - green
    color = f"rgb({red},{green},0)"
    bar_html = f'''
    <div style="background:#e0e0e0; border-radius: 8px; width: 100%; height: 20px; margin-bottom: 5px;">
      <div style="width: {score}%; background: {color}; height: 100%; border-radius: 8px;"></div>
    </div>
    '''
    return bar_html

In [19]:
def simple_pronunciation_feedback(user_text, target_text):
    sm = SequenceMatcher(None, user_text, target_text)
    ratio = sm.ratio()
    score = int(ratio * 100)
    if ratio > 0.85:
        verbal_feedback = "Great job! Your pronunciation is very clear and sounds almost native."
    elif ratio > 0.7:
        verbal_feedback = "Good effort! Some words could be clearer. Try saying each word a bit slower."
    elif ratio > 0.5:
        verbal_feedback = "Keep practicing. A few words are hard to understand. Break the sentence into parts and speak them one at a time."
    else:
        verbal_feedback = "Try again. Say the sentence slowly, focusing on each word and letter."
    return verbal_feedback, score

In [20]:
def simple_tone_feedback(pitch_mean, pitch_std, energy_mean):
    feedback = ["### Tone & Intonation Feedback"]
    if 100 <= pitch_mean <= 130:
        feedback.append(random.choice([
            "Your voice sounds warm and caring, suitable for nursing.",
            "Nice warmth in your voice, very professional."
        ]))
    elif pitch_mean < 100:
        feedback.append(random.choice([
            "Your voice is a bit low. Try speaking with a slightly brighter tone.",
            "Try raising your voice slightly to sound more friendly."
        ]))
    else:
        feedback.append(random.choice([
            "Your voice is a bit high-pitched. Soften your tone for a calmer effect.",
            "Try to relax your pitch to sound more soothing."
        ]))
    if pitch_std < 8:
        feedback.append("Your intonation is a little flat. Add gentle rises and falls to sound more engaging.")
    else:
        feedback.append(random.choice([
            "Nice intonation! You sound empathetic and natural.",
            "Great variation in your tone, very comforting."
        ]))
    if energy_mean < 0.03:
        feedback.append("Try speaking a little louder so you're easy to hear.")
    elif energy_mean > 0.08:
        feedback.append("Your voice is a bit loud—soften a little for a gentle bedside manner.")
    else:
        feedback.append("Your speaking volume is just right.")
    return "\n".join(feedback)

In [21]:
def analyze_tone(audio_path):
    y, sr = librosa.load(audio_path, sr=22050)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_vals = [pitches[np.argmax(magnitudes[:, t]), t] for t in range(magnitudes.shape[1]) if magnitudes[:, t].max() > 0.1]
    pitch_mean = np.mean(pitch_vals) if pitch_vals else 0
    pitch_std = np.std(pitch_vals) if pitch_vals else 0
    rms = librosa.feature.rms(y=y)[0]
    energy_mean = np.mean(rms)
    return simple_tone_feedback(pitch_mean, pitch_std, energy_mean)

In [22]:
def practice_pronunciation_with_history(audio_path, selected_phrase, prev_score):
    if audio_path is None:
        return "", "No recording found. Please record your voice.", "", None, prev_score

    result = model.transcribe(audio_path, language="de", task="transcribe", verbose=False)
    transcript = result["text"].strip()

    user_text = normalize_text(transcript)
    target_text = normalize_text(selected_phrase)

    verbal_feedback, score = simple_pronunciation_feedback(user_text, target_text)
    tone_feedback = analyze_tone(audio_path)
    celebration = CELEBRATION_GIF if score > 85 else None

    if prev_score == 0:
        comparison_text = "This is your first attempt."
        prev_bar = ""
    else:
        if score > prev_score:
            comparison_text = f"⬆️ Improvement! Previous: {prev_score}%, Current: {score}%"
        elif score < prev_score:
            comparison_text = f"⬇️ Score dropped. Previous: {prev_score}%, Current: {score}%"
        else:
            comparison_text = f"⚖️ Same score as previous: {score}%"
        prev_bar = similarity_score_bar(prev_score)

    curr_bar = similarity_score_bar(score)

    feedback_html = f"""
    <div>
      <p>{verbal_feedback}</p>
      <p><b>Previous score:</b> {prev_score}%</p>
      {prev_bar}
      <p><b>Current score:</b> {score}%</p>
      {curr_bar}
      <b>Comparison:</b> {comparison_text}
    </div>
    """

    return transcript, feedback_html, tone_feedback, celebration, score

In [23]:
def restart_recording():
    return None


In [24]:
with gr.Blocks(theme=gr.themes.Soft(), title="Nurse German Pronunciation") as demo:
    gr.Markdown("# 🇩🇪 German Pronunciation Coach for Nurses\nPractice key phrases. Get plain feedback to sound clear and caring.")

    phrase_dropdown = gr.Dropdown(choices=list(PHRASES.keys()), value="Wie geht es Ihnen?", label="🗣️ Select a Phrase")

    with gr.Row():
        gr.Markdown("### 🔉 Listen to Example")
    audio_player = gr.Audio(label="Native Pronunciation", visible=True)
    play_btn = gr.Button("🎧 Play Example", variant="primary")

    with gr.Row():
        gr.Markdown("### 🎤 Record Your Voice")
    record_btn = gr.Microphone(label="Click to Record", type="filepath", render=True)
    restart_btn = gr.Button("🔄 Record Again", variant="secondary")

    transcript_out = gr.Textbox(label="🗣️ Your Speech (Transcribed)")
    feedback_out = gr.Markdown(label="Pronunciation Feedback")
    tone_out = gr.Markdown(label="Tone & Intonation")
    celebration_img = gr.Image(label="🎉 Celebration", visible=False)

    previous_score = gr.State(0)

    play_btn.click(fn=play_audio, inputs=phrase_dropdown, outputs=audio_player)
    record_btn.change(
        fn=practice_pronunciation_with_history,
        inputs=[record_btn, phrase_dropdown, previous_score],
        outputs=[transcript_out, feedback_out, tone_out, celebration_img, previous_score]
    )
    restart_btn.click(fn=restart_recording, inputs=[], outputs=[record_btn])

In [25]:
demo.launch(share=True)
#please click the link to get better view in browser

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://425454d5852348bf8a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
